In [177]:
import numpy as np
from scipy.stats import logistic

### Код для Задачи 2(а)

In [178]:
scale = 1
n = 100
m = 100000
alpha = 1
alpha_1 = 1/alpha
w = np.random.normal(0, scale, n)
x = np.random.normal(0, np.sqrt(alpha_1), (m, n))
def p(x):
    return logistic.cdf(w @ x)
y, y_pred = [], []
P, N = 0,0
for i in range(m):
    y_i = np.random.binomial(1, p(x[i]))
    y += [y_i]
    y_pred += [p(x[i])]
    if(y_i == 1):
        P += 1
    else:
        N += 1
y = np.array(y)

Y = np.stack((y_pred, y), axis=1)
Y = Y[np.argsort(Y[:, 0])]
Y = np.flipud(Y)
TP, FP, TN, FN = 0, 0, 0, 0 
last_TP = 0
res = 0
for i in range(len(Y)):
    if (Y[i][1] == 0):
        FP += 1
        res += TP
        last_TP = TP
    else:
        TP += 1
print(FP , TP)
print(N , P)
print(res / (N * P))


50142 49858
50142 49858
0.9900392708607431


Семплированием для m стремящегося к бесконечности получается AUC = 1, что совпадает с теоретической оценкой

### Код для Задачи 2(б)

In [423]:
scale = 1
n = 100
m = 10
alpha = 0.5
alpha_1 = 1/alpha
k = 100
ans = []
for i in range(k):
    w = np.random.normal(0, np.sqrt(alpha_1), n)
    x = np.random.normal(0, scale, (m, n))


    y, y_pred = [], []

    def p(x):
        return logistic.cdf(w @ x)
    P, N = 0, 0
    for i in range(m):
        y_i = np.random.binomial(1, p(x[i]))
        if(y_i == 1):
            P += 1
        else:
            N += 1
        y += [y_i]
    w = np.random.normal(0, np.sqrt(alpha_1), n)
    for i in range(m):
        y_pred += [p(x[i])]
    y = np.array(y)

    Y = np.stack((y_pred, y), axis=1)
    Y = Y[np.argsort(Y[:, 0])]
    Y = np.flipud(Y)

    TP, FP, TN, FN = 0, 0, 0, 0 
    last_TP = 0
    res = 0
    for i in range(len(Y)):
        if (Y[i][1] == 0):
            FP += 1
            res += TP
            last_TP = TP
        else:
            TP += 1
    ans += [res / (N * P)]
ans = np.mean(ans)
print(ans)

0.4803904761904762


Семплированием для любого m получается AUC = 0.5С, что совпадает с теоретической оценкой

### Код для Задачи 4(в)

In [415]:
from scipy.stats import t
scale = 1
n = 100
m1 = 100
m2 = 100
alpha = 0.01
x1 = np.random.normal(0, scale, (m1, n))
x2 = np.random.normal(0, scale, (m2, n))
jd = 0
x1[..., jd] += 1
TP, FP, TN, FN = 0, 0, 0, 0 


def crit(x1, x2, m1, m2, i):
    x1_mean = np.mean(x1[:, i])
    x2_mean = np.mean(x2[:, i])
    x1_std = np.std(x1[:, i], ddof = 1)
    x2_std = np.std(x2[:, i], ddof = 1)
    #print(x2[:, i])
    s2 = np.sqrt(((m1 - 1) * x1_std**2  + (m2 - 1) * x2_std**2)/ (m1 + m2 - 2))
    return np.sqrt(m1*m2 / (m1 + m2 - 2)) * (x1_mean - x2_mean) / s2
p = []
T = []
for i in range(n - 1, -1, -1):
    T += [np.abs(crit(x1, x2, m1, m2, i))]
    pi = t(df = m1 + m2 - 2).cdf(np.abs(crit(x1, x2, m1, m2, i)))
                       
    p +=  [2*(1 - pi)]

T.sort()
T.reverse()
p.reverse()
print(T)

p = np.array(p)
h = p.argsort()
p = p[h]

for i in range(n):

    print(((i+1) * alpha) / n)
    if(p[i] >= ((i+1) * alpha) / n):
        FN = i - TN
        break
    if(h[i] == jd):
        TN += 1
FP += 1 - TN
TP = n - 1 - FN
print((FP, TP))

[9.129114006530543, 2.470879781871731, 2.355864615250967, 2.315255010420582, 2.221624745074848, 2.046277825718322, 1.9936102316774642, 1.8933915514296478, 1.8530771472075178, 1.8002736076565564, 1.7853005048915538, 1.7677488487025956, 1.7168449792438387, 1.7081153244164111, 1.7029001841529956, 1.6277887969965288, 1.625158652939335, 1.5713123821385468, 1.4873305152979999, 1.3507967169175001, 1.283163967843225, 1.2752910874128924, 1.2563524304855263, 1.179936222580228, 1.1313981857984743, 1.093518029223394, 1.0916002215947584, 1.0909692243840965, 1.0897547322876346, 1.0881565978836458, 1.0779179631949982, 1.0715106666547551, 1.0473393584410988, 1.0473329126308082, 1.0449864600401155, 1.0316407911566858, 1.0049948943560476, 0.9897847995426361, 0.9631269110786483, 0.9201070701893168, 0.920013542921821, 0.8932333749364241, 0.8884605091407898, 0.8864969781638442, 0.850648427808152, 0.812405920195407, 0.8037436680689668, 0.782313681894639, 0.7683895932783591, 0.7433685069568285, 0.74004123861

Если тестировать гипотезы по отдельности, то гипотеза о том, что 1ый признак с одинаковым матожиданием отвергается, для всех она принимается в большинстве случаев, но с большим числом m1 и m2 они принимаются правктически всегда.

Если тестировать гипотезы совместно, то вне зависимости  от m1 и m2 гипотеза для 1го признака отвергается а для всех остальных принимается.